**第10章**

In [1]:
from myABBR import * ;import myUtil as mu ; np.set_printoptions(precision=6) 
# 1.初期値 bP:buyProtection
tradeDT = jDT(2022,9,19) ;                          setEvDT(tradeDT)
matDT,           ntlAMT,   rfRT,  cdsQT,   hzdRT                    =\
jDT(2023,9,20),10_000_000, 0.10, 1.30/100,  0;  hzdRT=cdsQT/(1-rcvRTj)

# 2.cdsスケジュール
cdsSCD = ql.Schedule(tradeDT+1, matDT, pdFreqQ, calWK, FLLW, unADJ, dtGENf, EoMf)

# 3.DF,hazardRateカーブ
dsCvOBJ, dsCvHDL = mu.ffTSH(tradeDT,           rfRT, dcA365)
hzCvOBJ = ql.FlatHazardRate(tradeDT,mu.sqHDL(hzdRT), dcA365)
hzCvHDL = ql.DefaultProbabilityTermStructureHandle(hzCvOBJ)
print(f'(満期日) DF:{dsCvOBJ.discount           (matDT):.6f}, '
        f'生存確率Q:{hzCvOBJ.survivalProbability(matDT):.6%}, '
        f'ﾊｻﾞｰﾄﾞﾚｰﾄ:{hzCvOBJ.hazardRate         (matDT):.6%}' ) 

(満期日) DF:0.904590, 生存確率Q:98.014497%, ﾊｻﾞｰﾄﾞﾚｰﾄ:2.000000%


In [2]:
# 4.cds NPV計算
cdsOBJ   = ql.CreditDefaultSwap(bP, ntlAMT, cpn100, cdsSCD, FLLW, dcA360)
midENG   = ql.MidPointCdsEngine(hzCvHDL, rcvRTj, dsCvHDL)
cdsOBJ.setPricingEngine(midENG)
impHzdRT = cdsOBJ.impliedHazardRate(cdsOBJ.NPV(), dsCvHDL, dcA365, rcvRTj)

# 5.プリント
print('-'*14,'\n' 
      f'tradeDate     :{cdsOBJ.tradeDate().ISO():}    \n' 
      f'couponLegNPV  :{cdsOBJ.couponLegNPV()   :,.2f}\n' 
      f'defaultLegNPV :{cdsOBJ.defaultLegNPV()  :,.2f}\n' 
      f'NPV           :{cdsOBJ.NPV()            :,.2f}\n' 
      f'(Checking     :{cdsOBJ.couponLegNPV()+cdsOBJ.defaultLegNPV():,.2f})')
print('-'*14,'\n' 
      f'imp.HazardRate:{impHzdRT                :.6%}  \n'  
      f'fairSpread    :{cdsOBJ.fairSpread()     :.6%}'   )

-------------- 
tradeDate     :2022-09-19    
couponLegNPV  :-94,323.10
defaultLegNPV :122,462.50
NPV           :28,139.40
(Checking     :28,139.40)
-------------- 
imp.HazardRate:2.000000%  
fairSpread    :1.298330%


In [3]:
# hazardRrate計算 (エンジン不要)
npv0OBJ = ql.CreditDefaultSwap(bP, ntlAMT, cdsQT, cdsSCD, FLLW, dcA360)
impHzdRT= npv0OBJ.impliedHazardRate(0.0, dsCvHDL, dcA365, rcvRTj)
print(f'impHzdRT:{impHzdRT:.5%}')

impHzdRT:2.00257%


In [4]:
# 6.キャッシュフロー表: cdsOBJ.coupons()を使用
tradeDT, ntlAMT = cdsOBJ.tradeDate(), cdsOBJ.notional() #関数化の為 設定
dfCDS = pd.DataFrame({
    'payDate':    cpn.date(),
    'coupon':     cpn.rate(),
    'accStt':     cpn.accrualStartDate().ISO(),
    'accEnd':     cpn.accrualEndDate(),
    'days':       cpn.amount()/(ntlAMT*cpn.rate()    /360),
    'YF':         cpn.amount()/(ntlAMT*cpn.rate()*365/360),
    'amount':     cpn.amount(),
    } for cpn in map(ql.as_coupon, cdsOBJ.coupons()))
# 起算日の挿入
dfEFF = pd.DataFrame([{'payDate': cdsOBJ.protectionStartDate(), 
                        'accEnd': cdsOBJ.protectionStartDate()}],columns=dfCDS.columns)
dfCDS = pd.concat([dfEFF, dfCDS], ignore_index=True)            
#ディスカウントファクター
psDF  = [1.0                  for dt in dfCDS.accEnd if dt <= tradeDT] #past   DF
fuDF  = [dsCvOBJ.discount(dt) for dt in dfCDS.accEnd if tradeDT < dt ] #future DF
dfCDS = pd.concat([dfCDS, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
#生存確率QとdQ
psQ   = [1.0                             for dt in dfCDS.accEnd if dt <= tradeDT]
fuQ   = [hzCvOBJ.survivalProbability(dt) for dt in dfCDS.accEnd if tradeDT < dt ]
dfCDS = pd.concat([dfCDS, pd.DataFrame(psQ+fuQ, columns=['Q']) ], axis=1) 
dQ    = np.insert(np.diff(dfCDS.Q)*(-1), 0, 0) 
dfCDS = pd.concat([dfCDS, pd.DataFrame( dict(dQ=dQ)) ], axis=1) 
# mid計算
midDTs  = dfCDS.payDate[:-1]+(np.diff(dfCDS.payDate)/2).astype('int')
midDFs  = [dsCvOBJ.discount(dd) for dd in midDTs]
midQs   = [hzCvOBJ.survivalProbability(dd) for dd in midDTs]
dfDFQm  = pd.DataFrame( dict(mDate=midDTs, mDF=midDFs, mQ=midQs))
dfDFQm.mDate = dfDFQm.mDate.map(lambda x: x.ISO())
dfDFQm  = pd.concat([pd.DataFrame(columns=['mDate'],index=[0]),     #空の行を追加
                                                    dfDFQm],ignore_index=True)
dfCDS   = pd.concat([dfCDS, dfDFQm ], axis=1)  
#日付修正(ISOフォーマット)
dfCDS.payDate =  dfCDS.payDate.map(lambda x:x.ISO())
dfCDS.accEnd  =  dfCDS.accEnd.map(lambda x:x.ISO())
display(dfCDS[:].style.format(fmtSCF))

,payDate,coupon,accStt,accEnd,days,YF,amount,DF,Q,dQ,mDate,mDF,mQ
0,2022-09-20,nan%,nan,2022-09-20,nan,nan,nan,0.99972606,0.999945,0.000000,nan,nan,nan
1,2022-12-20,1.000000%,2022-09-20,2022-12-20,91,0.249315,"25,277.78",0.97510953,0.994972,0.004974,2022-11-04,0.987476,0.997483
2,2023-03-20,1.000000%,2022-12-20,2023-03-20,90,0.246575,"25,000.00",0.95135974,0.990077,0.004895,2023-02-03,0.963161,0.992521
3,2023-06-20,1.000000%,2023-03-20,2023-06-20,92,0.252055,"25,555.56",0.92767994,0.985098,0.004979,2023-05-05,0.939445,0.987585
4,2023-09-20,1.000000%,2023-06-20,2023-09-20,92,0.252055,"25,555.56",0.90458955,0.980145,0.004953,2023-08-05,0.916062,0.982619


In [5]:
# couponLeg, DefaultLeg, NPV
hcCpnLEG = -(dfCDS.amount * dfCDS.DF * dfCDS.mQ).sum()
hcDftLEG =  (1-rcvRTj)    *(dfCDS.mDF* dfCDS.dQ).sum()
hcNPV    = hcCpnLEG + hcDftLEG*ntlAMT
print(f'(hc)cpnLegNPV: {hcCpnLEG:,.2f}',f'(hc)dftLegNPV: {hcDftLEG*ntlAMT:,.2f} ',
      f'(hc)NPV: {hcNPV:,.2f}')

# RPV01, fairSPD, Hazardrate
hcRPV01  =  (dfCDS.YF     * dfCDS.DF * dfCDS.mQ).sum()
hcFairSPD= (hcDftLEG/(hcRPV01*365/360))
hcHzdRT  = np.log(dfCDS.mQ.iloc[-2]/dfCDS.mQ.iloc[-1])/dfCDS.YF.iloc[-1]
print(f'(hc)RPV01: {hcRPV01:.8f}    ',  f'(hc)fairSPD: {hcFairSPD:.8%}  ',
      f'(hc)hzdRT: {hcHzdRT:.8%}')

(hc)cpnLegNPV: -94,321.19 (hc)dftLegNPV: 122,462.50  (hc)NPV: 28,141.31
(hc)RPV01: 0.93029116     (hc)fairSPD: 1.29835622%   (hc)hzdRT: 2.00000000%


In [6]:
# Sony 5年CDS T/D 2022/8/19 buyProtection
from myABBR import * ; import myUtil as mu 
# 1.初期値
tradeDT,         cdsTNR,   cdsQT,     ntlAMT                              =\
jDT(2022,8,19), pD('5y'), 30/10000, 10_000_000.0; setEvDT(tradeDT)
    #春分/秋分の休日追加
jpnHLDY = [jDT(yyyy, 3, 20) for yyyy in range(2024, 2027)]\
         +[jDT(2027,3,22)]+[jDT(2026,9,21)]+[jDT(2026,9,22)]+[jDT(2026,9,23)]
for dt in jpnHLDY: calWK.addHoliday(dt)
    #満期日
matDT   = ql.cdsMaturity(tradeDT, cdsTNR, dtGEN15); print('matDT:',matDT.ISO())

# 2.cdsスケジュール
cdsSCD  = ql.Schedule(tradeDT, matDT, pdFreqQ, calWK, FLLW, unADJ, dtGEN15, EoMf)
print('CDSスケジュール:',list(cdsSCD)[:3], '...\n', ' '*15, list(cdsSCD)[-3:] )

# 3.TONAカーブ作成
crvDATA = [('depo','1d',-0.009), ('swap','1m',-0.01807),('swap','6m',-0.01043),
           ('swap','12m',0.0125),('swap','18m',0.03125),('swap','2y',0.04875),
           ('swap','3y',0.07375),('swap','5y',0.11854), ('swap','7y',0.19146)]
_, dsCvOBJ, dsCvHDL, _ = mu.makeTonaCurve(crvDATA)
    #hazardRate計算 (エンジン不要)
npv0OBJ = ql.CreditDefaultSwap(bP,ntlAMT,cdsQT,cdsSCD,FLLW,dcA360,
                                   True,True,tradeDT,ql.FaceValueClaim(),dcA360t)
hzdRT   = npv0OBJ.impliedHazardRate(0.0,dsCvHDL,dcA365,rcvRTj,1e-10,
                                                       ql.CreditDefaultSwap.ISDA)
hzCvOBJ = ql.FlatHazardRate(0, calWK, mu.sqHDL(hzdRT), dcA365)
hzCvHDL = ql.DefaultProbabilityTermStructureHandle(hzCvOBJ)

# 4.cds NPV計算
cdsOBJ  = ql.CreditDefaultSwap(bP,ntlAMT,cpn100,cdsSCD,FLLW,dcA360,
                                True,True,tradeDT, ql.FaceValueClaim(), dcA360t)
isdaENG = ql.IsdaCdsEngine(hzCvHDL,rcvRTj,dsCvHDL)
cdsOBJ.setPricingEngine(isdaENG)
impHzdRT  =cdsOBJ.impliedHazardRate(cdsOBJ.NPV(),dsCvHDL,dcA365,rcvRTj)
netTwoLEG =cdsOBJ.couponLegNPV()+cdsOBJ.defaultLegNPV()+cdsOBJ.accrualRebateNPV()
settleAMT =cdsOBJ.fairUpfront()*ntlAMT+cdsOBJ.accrualRebate().amount()*-1
# 5.プリント
print('-'*14,'\n' 
    f'tradeDate     :{cdsOBJ.tradeDate().ISO()  :}     \n'      
    f'couponLegNPV  :{cdsOBJ.couponLegNPV()     : ,.2f}\n' 
    f'accrRbtNPV    :{cdsOBJ.accrualRebateNPV() : ,.2f}\n' 
    f'defaultLegNPV :{cdsOBJ.defaultLegNPV()    : ,.2f}\n' 
    f'NPV           :{cdsOBJ.NPV()              : ,.2f}\n' 
    f'(Checking     :{netTwoLEG                 : ,.2f})')
print('-'*14, '\n'
    f'imp.HazardRate:{impHzdRT                  : .6%}\n'
    f'fairSpread    :{cdsOBJ.fairSpread()       : .6%}'  )
print('-'*14, '\n'
    f'UpfrontDate   :{cdsOBJ.upfrontPayment().date().ISO() :}  \n'   
    f'fairUpfront(%):{cdsOBJ.fairUpfront()              : ,.8%}\n' 
    f'UpfntAMT      :{cdsOBJ.fairUpfront()*ntlAMT       : ,.2f}\n' 
    f'accrualRbtAMT :{cdsOBJ.accrualRebate().amount()*-1: ,.2f}\n' 
    f'(accrual days :{cdsOBJ.accrualRebate().amount()*360/(cpn100*ntlAMT):,.0f}日)\n'
    f'settleAMT     :{settleAMT                         : ,.2f}' )
# 6.キャッシュフロー表
dfCDS = mu.cdsCashFlow(cdsOBJ, hzCvOBJ, dsCvOBJ)
display(dfCDS.style.format(fmtSCF))

matDT: 2027-06-20
CDSスケジュール: [Date(20,6,2022), Date(20,9,2022), Date(20,12,2022)] ...
                 [Date(21,12,2026), Date(23,3,2027), Date(20,6,2027)]
-------------- 
tradeDate     :2022-08-19     
couponLegNPV  :-501,001.58
accrRbtNPV    : 16,944.47
defaultLegNPV : 145,217.13
NPV           :-338,839.97
(Checking     :-338,839.97)
-------------- 
imp.HazardRate: 0.467847%
fairSpread    : 0.300000%
-------------- 
UpfrontDate   :2022-08-24  
fairUpfront(%):-3.38839388%
UpfntAMT      :-338,839.39
accrualRbtAMT :-16,944.44
(accrual days :61日)
settleAMT     :-355,783.83


,payDate,coupon,accStt,accEnd,days,YF,amount,DF,Q,dQ,mDate,mDF,mQ
0,2022-08-19,nan%,nan,2022-08-19,nan,nan,nan,1.00000000,1.000000,0.000000,nan,nan,nan
1,2022-09-20,1.000000%,2022-06-20,2022-09-20,92,0.252055,"25,555.56",1.00001510,0.999590,0.000410,2022-09-04,1.000007,0.999795
2,2022-12-20,1.000000%,2022-09-20,2022-12-20,91,0.249315,"25,277.78",1.00003835,0.998425,0.001165,2022-11-04,1.000027,0.999013
3,2023-03-20,1.000000%,2022-12-20,2023-03-20,90,0.246575,"25,000.00",1.00003039,0.997273,0.001151,2023-02-03,1.000049,0.997849
4,2023-06-20,1.000000%,2023-03-20,2023-06-20,92,0.252055,"25,555.56",0.99993948,0.996098,0.001175,2023-05-05,0.999985,0.996685
5,2023-09-20,1.000000%,2023-06-20,2023-09-20,92,0.252055,"25,555.56",0.99982421,0.994924,0.001174,2023-08-05,0.999894,0.995511
6,2023-12-20,1.000000%,2023-09-20,2023-12-20,91,0.249315,"25,277.78",0.99965511,0.993764,0.001160,2023-11-04,0.999741,0.994350
7,2024-03-21,1.000000%,2023-12-20,2024-03-21,92,0.252055,"25,555.56",0.99946127,0.992593,0.001171,2024-02-04,0.999570,0.993178
8,2024-06-20,1.000000%,2024-03-21,2024-06-20,91,0.249315,"25,277.78",0.99920537,0.991436,0.001157,2024-05-05,0.999335,0.992020
9,2024-09-20,1.000000%,2024-06-20,2024-09-20,92,0.252055,"25,555.56",0.99893075,0.990267,0.001169,2024-08-05,0.999076,0.990851


In [7]:
# couponLeg, DefaultLeg, NPV
hcCpnLEG  = -(dfCDS.amount * dfCDS.DF * dfCDS.mQ).sum()
hcDftLEG  =  (1-rcvRTj)    *(dfCDS.mDF* dfCDS.dQ).sum()
accrRBT = cdsOBJ.accrualRebateNPV()
hcNPV   = hcCpnLEG + hcDftLEG*ntlAMT + accrRBT 
print(f'(hc)cpnLegNPV: {hcCpnLEG:,.2f} ', 
      f'(hc)dftLegNPV: {hcDftLEG*ntlAMT:,.2f} ',f'(hc)NPV: {hcNPV:,.2f}')

# RPV01, fairSPD :  (92-61)/365は初回利払いのYFを修正
hcRPV01  = (92-61)/365  * (dfCDS.DF.iloc[1] * dfCDS.mQ.iloc[1])  
hcRPV01 += (dfCDS.YF[2:]*  dfCDS.DF[2:]     * dfCDS.mQ[2:]    ).sum()
hcFairSPD= (hcDftLEG/(hcRPV01*365/360))
print(f'(hc)RPV01: {hcRPV01:.8f}      ',  f'(hc)fairSPD: {hcFairSPD:.8%}')

(hc)cpnLegNPV: -500,990.24  (hc)dftLegNPV: 145,217.37  (hc)NPV: -338,828.39
(hc)RPV01: 4.77418202       (hc)fairSPD: 0.30000552%
